#### GISC 425 T1 2020
# **LAB ASSIGNMENT 2**

In [ ]:
# You need to run this cell to get things setup
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import geopandas
import shapely

import math

A key capability of any programming language is iteration over collections of items. 

Before attempting this assignment you need to be up to speed with [iteration](01-iteration.ipynb), [strings](02-strings.ipynb), and [lists](03-lists.ipynb) in Python. These topics are covered in Chapters 7, 8, and 10 of [Think Python](https://greenteapress.com/wp/think-python-2e/). You should already have read these chapters and worked through those notebooks. If you haven't done that, stop reading this notebook and complete those instead. It will also be useful to understand how to open and read files. This is discussed in Chapter 14 of the book, but is also covered in this notebook, before we proceed to the assignment itself. 

## Tissot's indicatrix
This assignment explores this topic using [Tissot's indicatrix](https://en.wikipedia.org/wiki/Tissot's_indicatrix) as an example.

Tissot's indicatrix visualizes the distortions in a project by showing how a circle of equal area appears in that projection across a set of locations.  Here's an example

<img src="Mercator.png" width=50%>

This shows how the Mercator projection exaggerates area at high latitudes. Each circle on the map is the same area on Earth's surface, but those at high latitudes appear considerably larger. Depending on the projection this method also allows us to see how shapes are distorted by a projection.

## Making a geodesic circle
To do this, we are going to need to make equal area circles. I'm calling these `geodesic circles` because these are circles on the surface of a sphere whose radii are measured along the surface of the sphere, not in simple $(x,y)$ coordinates. I have provided code for this below.

In [ ]:
# returns a lon-lat point (in degrees) given
# a starting lon-lat and bearing and distance (all in radians)
# equation is from
# https://stackoverflow.com/questions/7222382/get-lat-long-given-current-point-distance-and-bearing
def lon_lat_at_bearing_and_distance(p, b, d):
    lon1 = p[0]
    lat1 = p[1]
    lat2 = math.asin(math.sin(lat1) * math.cos(d) + math.cos(lat1) * math.sin(d) * math.cos(b))
    lon2 = lon1 + math.atan2(math.sin(b) * math.sin(d) * math.cos(lat1), math.cos(d) - math.sin(lat1) * math.sin(lat1))
    # convert result to degrees before returning
    return (math.degrees(lon2), math.degrees(lat2))


# makes a shapely.geometry in lon-lat coordinates
# based on a provide centre point in lon-lat and 
# radius expressed in degrees
def geodesic_circle(p=(0,0), rd=2):
    # extract the numbers and convert to radians
    # assume p is a (lon, lat) tuple in degrees
    ll = (math.radians(p[0]), math.radians(p[1]))
    # convert rd to degrees
    rr = math.radians(rd) 
    
    # empty list for the resulting points
    pts = []
    # 360 in 1 degree steps
    # each time get a point at the distance rr away in that direction
    for bearing in range(360):
        bearing_r = math.radians(bearing)
        pts.append(lon_lat_at_bearing_and_distance(ll, math.radians(bearing), rr))
    return shapely.geometry.Polygon(pts)


Check it works by making some geodesic circles in the cell below.  Since they are `shapely.geometry` objects they will plot. Change the centre point to see how the shape varies when they are plotted in simple longitude-latitude space.

In [ ]:
# try setting different center lon-lat coordinates, by
# providing various values for the `p` parameter
# Note the p is in the order lon, lat, so Wellington would be p=(174, -41) or thereabouts
geodesic_circle()

## Making a GeoDataFrame from a `list` of geodesic circles
To plot a Tissot's indicatrix we will make a `GeoDataFrame` from a `list` of geodesic circles, and plot them on top of a map of the world. Below is a function to make a `GeoDataFrame` from a list of `geodesic_circle`s. To show how this works, I'll just make a small list so you can see the idea.

In [ ]:
def make_gdf_from_circles(circles):
    gs = geopandas.GeoSeries(circles)
    gdf = geopandas.GeoDataFrame(geometry=gs)
    gdf.crs = "EPSG:4326"
    return gdf

Here's some code to make a `GeoDataFrame` from a list of circles using this function.

In [ ]:
circle_list = [geodesic_circle(), 
               geodesic_circle(p=(0,30)), 
               geodesic_circle(p=(0,60))
              ]
tissot = make_gdf_from_circles(circle_list)

And finally, here's how we can make a visualization. First grab the built in world dataset:

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

And now assemble the map

In [ ]:
fig = pyplot.figure(figsize=(12,6))
base = fig.add_subplot(111)

world.plot(ax=base, facecolor='lightgrey')
tissot.plot(ax=base, facecolor='r', alpha=0.5)

This is obviously not a very useful example, because it only shows three circles. 

That's the point of the assignment: to make more extensive sets of circles that will assist in the visualization.

## Here's one I prepared earlier
The first approach to this will make use of a list of centres that I made using a package in *R* called [`dggridR` which implements discrete global grids](https://github.com/r-barnes/dggridR).  I have output the resulting equally space grid centres to a CSV file `cc.csv`.

Reading this file gives us a chance to learn about opening and reading files in Python.  This is surprisingly easy.

In [ ]:
# The open() function opens the specified file for reading
cc_file = open("cc.csv")
# The readlines() method reads the file contents into a variable, as a list, one item per line
data = cc_file.readlines()

# now we have the data we should close the file
cc_file.close()

# take a look at the first few items in the list (note the slice operation)
data[:10]

## **The first part of the assignment is in this section**
So now we have the list of circle centres. They are in a list (which means we can read through them, *but* it is a list of strings, and the strings also have a `\n` newline character at the end. To make the list of circles we need, we'll have to read through this list of strings, and parse each one into a pair of coordinates to be passed to the `geodesic_circle` function.

The comments in the cell below guide you through how to do this.

You should complete the cell to make a list of `geodesic_circle`s.

In [ ]:
# first make an empty list for the circles
# call it something appropriate


# loop through the data read from the file using a for operation
# you can skip the first line with a slice operation


# inside the loop, with each line of the data
# you need to remove the last character (the newline character)

# then you need to split the line into two strings at the ',' separator

# then you should have a list of two items, the lon and lat, as strings
# you need to convert these to float values and then pass them into
# the geodesic_circle function as the p parameter

# then append the resulting geodesic_circle to the list of circles


Now you have a list of circles, make a `GeoDataFrame` from it using the `make_gdf_from_circles()` function.

In [ ]:
# Put a line of code here to make a GeoDataFrame from your list of circles

And now make a plot, using the code from a cell that we ran a while back (copy and paste it into the cell below, and modify as required.

In [ ]:
# Copy and paste code from a cell above to make the map, 
# but using the GeoDataFrame of circles you made



## **The second part of the assignment is in this section**
Most Tissot indicatrix visualizations make use of a longitude-latitude grid of locations.

This will involve nested loops, which will be a good test of your understanding of iteration.

Again you will need to start with an empty list.

This time you will need two nested loops, one stepping through a series of longitude values, and the inner one stepping through a series of latitude values. It's a good idea to avoid longitude -180 or +180 and latitude -90 or +90 so consider that.

Also worth noting are the additional options you can pass to `range()`. Just to give you the idea:

In [ ]:
for x in range(10, 100, 20):
    print(x)

This should enable you to make outer and inner `for` loops that generate a suitable set of longitude and latitude values to generate `geodesic_circle`s to `append` to the empty `list` in the same way you did in the previous step.

Write code in the cell below.

In [ ]:
# Make an empty list for the circles


# Use nested for loops and the range operation to make a series of circles
# arranged in a longitude - latitude grid, appending them to the list each time


# then use make_gdf_from_circles() to make GeoDataFrame from your list of circles


# then use the code you used before to make a map

## Submission
Save a completed notebook with a name that includes your name. Upload to the dropbox provided on blackboard.

In the cell below you can provide any commentary you would like.